In [11]:
import os
import json
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
os.getcwd()

'C:\\Users\\Danie\\ITNPBD2-assignment-1'

In [5]:
os.chdir("C:\\Users\\Danie")

# Initial data exploration and cleaning

I started by loading the csv and json files into panda data frames to begin understanding the datasets.

From these initial observations, I began cleaning the data.

## Products.csv initial exploration

In [46]:
products_csv = pd.read_csv("products.csv")
display(products_csv.head())
print(products_csv.shape)
print(products_csv.count()) #Using the count function has showed that the SKU, Description, and Price columns are missing some data
print(products_csv.isnull().sum()) #Further to the .count() funtion, I am finding out how many values are missing from each column
print(products_csv.dtypes)


,Uniq_id,SKU,Name,Description,Price,Av_Score
0,b6c0b6bea69c722939585baeac73c13d,pp5006380337,Alfred Dunner® Essential Pull On Capri Pant,Youll return to our Alfred Dunner pull-on capr...,41.09,2.625
1,93e5272c51d8cce02597e3ce67b7ad0a,pp5006380337,Alfred Dunner® Essential Pull On Capri Pant,Youll return to our Alfred Dunner pull-on capr...,41.09,3.000
2,013e320f2f2ec0cf5b3ff5418d688528,pp5006380337,Alfred Dunner® Essential Pull On Capri Pant,Youll return to our Alfred Dunner pull-on capr...,41.09,2.625
3,505e6633d81f2cb7400c0cfa0394c427,pp5006380337,Alfred Dunner® Essential Pull On Capri Pant,Youll return to our Alfred Dunner pull-on capr...,41.09,3.500
4,d969a8542122e1331e304b09f81a83f6,pp5006380337,Alfred Dunner® Essential Pull On Capri Pant,Youll return to our Alfred Dunner pull-on capr...,41.09,3.125


(7982, 6)
Uniq_id        7982
SKU            7915
Name           7982
Description    7439
Price          5816
Av_Score       7982
dtype: int64
Uniq_id           0
SKU              67
Name              0
Description     543
Price          2166
Av_Score          0
dtype: int64
Uniq_id         object
SKU             object
Name            object
Description     object
Price          float64
Av_Score       float64
dtype: object


## Users.csv initial exploration

In [47]:
users = pd.read_csv("users.csv")
display(users.head())
print(users.shape)
print(users.isnull().sum()) #Checking for missing values
print(users.dtypes)

,Username,DOB,State
0,bkpn1412,31.07.1983,Oregon
1,gqjs4414,27.07.1998,Massachusetts
2,eehe1434,08.08.1950,Idaho
3,hkxj1334,03.08.1969,Florida
4,jjbd1412,26.07.2001,Georgia


(5000, 3)
Username    0
DOB         0
State       0
dtype: int64
Username    object
DOB         object
State       object
dtype: object


## Reviews.csv initial exploration

In [48]:
reviews_csv = pd.read_csv("reviews.csv")
display(reviews_csv.head())
print(reviews_csv.shape)
print(reviews_csv.isnull().sum())
print(reviews_csv.dtypes)

,Uniq_id,Username,Score,Review
0,b6c0b6bea69c722939585baeac73c13d,fsdv4141,2,You never have to worry about the fit...Alfred...
1,b6c0b6bea69c722939585baeac73c13d,krpz1113,1,Good quality fabric. Perfect fit. Washed very ...
2,b6c0b6bea69c722939585baeac73c13d,mbmg3241,2,I do not normally wear pants or capris that ha...
3,b6c0b6bea69c722939585baeac73c13d,zeqg1222,0,I love these capris! They fit true to size and...
4,b6c0b6bea69c722939585baeac73c13d,nvfn3212,3,This product is very comfortable and the fabri...


(39063, 4)
Uniq_id     0
Username    0
Score       0
Review      0
dtype: int64
Uniq_id     object
Username    object
Score        int64
Review      object
dtype: object


## jcpenney_products.json initial exploration

In [49]:
products_json = pd.read_json("jcpenney_products.json", lines = True)
display(products_json.head())
print(products_json.shape)
print(products_json.count())
print(products_json.isnull().sum())
print(products_json.dtypes)


,uniq_id,sku,name_title,description,list_price,sale_price,category,category_tree,average_product_rating,product_url,product_image_urls,brand,total_number_reviews,Reviews,Bought With
0,b6c0b6bea69c722939585baeac73c13d,pp5006380337,Alfred Dunner® Essential Pull On Capri Pant,You'll return to our Alfred Dunner pull-on cap...,41.09,24.16,alfred dunner,jcpenney|women|alfred dunner,2.625,http://www.jcpenney.com/alfred-dunner-essentia...,http://s7d9.scene7.com/is/image/JCPenney/DP122...,Alfred Dunner,8,"[{'User': 'fsdv4141', 'Review': 'You never hav...","[898e42fe937a33e8ce5e900ca7a4d924, 8c02c262567..."
1,93e5272c51d8cce02597e3ce67b7ad0a,pp5006380337,Alfred Dunner® Essential Pull On Capri Pant,You'll return to our Alfred Dunner pull-on cap...,41.09,24.16,alfred dunner,jcpenney|women|alfred dunner,3.000,http://www.jcpenney.com/alfred-dunner-essentia...,http://s7d9.scene7.com/is/image/JCPenney/DP122...,Alfred Dunner,8,"[{'User': 'tpcu2211', 'Review': 'You never hav...","[bc9ab3406dcaa84a123b9da862e6367d, 18eb69e8fc2..."
2,013e320f2f2ec0cf5b3ff5418d688528,pp5006380337,Alfred Dunner® Essential Pull On Capri Pant,You'll return to our Alfred Dunner pull-on cap...,41.09,24.16,view all,jcpenney|women|view all,2.625,http://www.jcpenney.com/alfred-dunner-essentia...,http://s7d9.scene7.com/is/image/JCPenney/DP122...,Alfred Dunner,8,"[{'User': 'pcfg3234', 'Review': 'You never hav...","[3ce70f519a9cfdd85cdbdecd358e5347, b0295c96d2b..."
3,505e6633d81f2cb7400c0cfa0394c427,pp5006380337,Alfred Dunner® Essential Pull On Capri Pant,You'll return to our Alfred Dunner pull-on cap...,41.09,24.16,view all,jcpenney|women|view all,3.500,http://www.jcpenney.com/alfred-dunner-essentia...,http://s7d9.scene7.com/is/image/JCPenney/DP122...,Alfred Dunner,8,"[{'User': 'ngrq4411', 'Review': 'You never hav...","[efcd811edccbeb5e67eaa8ef0d991f7c, 7b2cc00171e..."
4,d969a8542122e1331e304b09f81a83f6,pp5006380337,Alfred Dunner® Essential Pull On Capri Pant,You'll return to our Alfred Dunner pull-on cap...,41.09,24.16,view all,jcpenney|women|view all,3.125,http://www.jcpenney.com/alfred-dunner-essentia...,http://s7d9.scene7.com/is/image/JCPenney/DP122...,Alfred Dunner,8,"[{'User': 'nbmi2334', 'Review': 'You never hav...","[0ca5ad2a218f59eb83eec1e248a0782d, 9869fc8da14..."


(7982, 15)
uniq_id                   7982
sku                       7982
name_title                7982
description               7982
list_price                7982
sale_price                7982
category                  7982
category_tree             7982
average_product_rating    7982
product_url               7982
product_image_urls        7982
brand                     7982
total_number_reviews      7982
Reviews                   7982
Bought With               7982
dtype: int64
uniq_id                   0
sku                       0
name_title                0
description               0
list_price                0
sale_price                0
category                  0
category_tree             0
average_product_rating    0
product_url               0
product_image_urls        0
brand                     0
total_number_reviews      0
Reviews                   0
Bought With               0
dtype: int64
uniq_id                    object
sku                        object
name_title

In [44]:
products_json.replace("", np.nan, inplace=True)
print(products_json.isnull().sum()) #Finding out where missing values are

uniq_id                      0
sku                         67
name_title                   0
description                543
list_price                2166
sale_price                  18
category                   636
category_tree              636
average_product_rating       0
product_url                  0
product_image_urls         157
brand                        0
total_number_reviews         0
Reviews                      0
Bought With                  0
dtype: int64


In [39]:
reviews_json = pd.read_json("jcpenney_reviewers.json", lines=True)
display(reviews_json.head())
print(reviews_json.shape)
print(reviews_json.count())
print(reviews_json.isnull().sum())


,Username,DOB,State,Reviewed
0,bkpn1412,31.07.1983,Oregon,[cea76118f6a9110a893de2b7654319c0]
1,gqjs4414,27.07.1998,Massachusetts,[fa04fe6c0dd5189f54fe600838da43d3]
2,eehe1434,08.08.1950,Idaho,[]
3,hkxj1334,03.08.1969,Florida,"[f129b1803f447c2b1ce43508fb822810, 3b0c9bc0be6..."
4,jjbd1412,26.07.2001,Georgia,[]


(5000, 4)
Username    5000
DOB         5000
State       5000
Reviewed    5000
dtype: int64
Username    0
DOB         0
State       0
Reviewed    0
dtype: int64


In [45]:
reviews_json.replace("", np.nan, inplace=True)
print(reviews_json.isnull().sum()) #Finding out where missing values are

Username    0
DOB         0
State       0
Reviewed    0
dtype: int64


## Cleaning data sets and creating sub-sets

Cleaning products JSON file

In [57]:
products_json.replace("", np.nan, inplace=True)
print(products_json.isnull().sum())

print(products_json.dtypes)

uniq_id                      0
sku                         67
name_title                   0
description                543
list_price                2166
sale_price                 263
category                   636
category_tree              636
average_product_rating       0
product_url                  0
product_image_urls         157
brand                        0
total_number_reviews         0
Reviews                      0
Bought With                  0
dtype: int64
uniq_id                    object
sku                        object
name_title                 object
description                object
list_price                float64
sale_price                float64
category                   object
category_tree              object
average_product_rating    float64
product_url                object
product_image_urls         object
brand                      object
total_number_reviews        int64
Reviews                    object
Bought With                object
dtype: objec

In [52]:
products_json["list_price"] = products_json["list_price"].astype(float)

In [55]:
products_json["sale_price"] = pd.to_numeric(products_json["sale_price"], errors = "coerce") 
#When I initially ran the function to convert object to float, there was an error because some prices were in ranges. 

In [56]:
products_json["sale_price"] = products_json["sale_price"].astype(float)

Creating subset of Products JSON to show true average value by SKU

In [ ]:
subset = products_json["